# Know Your Music Taste (KYMT)
- Spotify Edition
- BETA.2.07.21

## Downloading and Importing neccessary libraries

In [1]:
!pip3 install spotipy
!pip3 install ColorThief

In [2]:
import spotipy as sp
from colorthief import ColorThief
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import PIL 
import urllib 
import sys
import io

## Custom functions

In [3]:
def playlistFeatures(results, sp_access):
  # create a list of song ids
  ids = []

  for item in results['tracks']['items']:
          track = item['track']['id']
          ids.append(track)
          
  song_meta = {'id':[],'album':[], 'name':[], 
               'artist':[],'explicit':[],'popularity':[]}

  for song_id in ids:
      # get song's meta data
      meta = sp_access.track(song_id)
      
      # song id
      song_meta['id'].append(song_id)

      # album name
      album = meta['album']['name']
      song_meta['album']+=[album]

      # song name
      song = meta['name']
      song_meta['name']+=[song]
      
      # artists name
      s = ', '
      artist=s.join([singer_name['name'] for singer_name in meta['artists']])
      song_meta['artist']+=[artist]
      
      # explicit: lyrics could be considered offensive or unsuitable for children
      explicit = meta['explicit']
      song_meta['explicit'].append(explicit)
      
      # song popularity
      popularity = meta['popularity']
      song_meta['popularity'].append(popularity)

  song_meta_df = pd.DataFrame.from_dict(song_meta)

  # check the song feature
  features = sp_access.audio_features(song_meta['id'])
  # change dictionary to dataframe
  features_df = pd.DataFrame.from_dict(features)

  # convert milliseconds to mins
  # duration_ms: The duration of the track in milliseconds.
  # 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
  features_df['duration_ms'] = features_df['duration_ms']/60000

  # combine two dataframe
  final_df = song_meta_df.merge(features_df)
  
  final_df.drop(['id', 'analysis_url', 'valence', 'time_signature', 
                 'track_href', 'uri', 'type', 'mode', 
                 'duration_ms', 'key'], 
                axis = 1, 
                inplace = True)

  subset_data = final_df.iloc[:, 4:]

  subset_data_scaled = MinMaxScaler().fit_transform(subset_data)

  subset_data_scaled = pd.DataFrame(subset_data_scaled, 
                                    columns = subset_data.columns)

  return subset_data_scaled

In [4]:
def playlistColor(results):

  if sys.version_info < (3, 0):
    from urllib2 import urlopen
  else:
      from urllib.request import urlopen

  img_url = ((results).get('images')[0]).get('url')

  color_thief = ColorThief(io.BytesIO(urlopen(img_url).read()))

  playlist_color = '#%02x%02x%02x' % color_thief.get_color(quality=1)    

  return playlist_color

In [5]:
client_id = '4d500ecf52a3447685e7389ca8a1dae9'
client_secret = 'c5b32293c6fa4339893431280cce8ac1'

sp_access = sp.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, 
                                                                           client_secret))

In [6]:
playlist_id_url = input('Enter Playlist Link:\n')

playlist_id = playlist_id_url.split('/')[4]

results = sp_access.playlist(playlist_id)

Enter Playlist Link:
https://open.spotify.com/playlist/37i9dQZF1DX7EF8wVxBVhG?si=7d7f2bed445c4120


In [7]:
playlist_data = playlistFeatures(results, sp_access)

In [8]:
playlist_data.head(3)

,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,tempo
0,1.000000,0.196379,0.138039,0.410160,0.033486,0.841137,0.786350,0.440994,0.191227
1,0.555556,0.389972,0.096755,0.466397,0.022463,0.719900,0.686944,0.518634,0.726834
2,0.518519,0.133705,0.078490,0.369632,0.047421,0.873537,0.833828,0.782609,0.216291


In [9]:
playlist_name = results.get('name')

## Main Plot

In [10]:
fig = go.Figure(
    data=go.Scatterpolar(
    r=np.int64(np.round(playlist_data.mean() * 10)),
    theta=playlist_data.columns.str.title(),
    fill='toself',
    line=dict(color = playlistColor(results)), 
    )
)

fig.update_layout(
    title = playlist_name+" Playlist's Features",
    font_size = 15, 
    showlegend=False,
    polar=dict(
        bgcolor = "rgb(241, 242, 246)",
        radialaxis=dict(visible=True),
        ),
)

# Add image
fig.add_layout_image(
    dict(
        source=((results).get('images')[0]).get('url'),
        xref="paper", yref="paper",
        x=1.05, y=0.25,
        sizex=0.8, sizey=0.8,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()